> # Neural Network Basic - Week3 과제


## Import Library

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from dataset.mnist import load_mnist

## Load Dataset
- MNIST 

In [2]:
(X_train, Y_train), (X_test, Y_test) = \
    load_mnist(normalize=True, one_hot_label=False)

In [3]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'Y_train shape: {Y_train.shape}')
print(f'Y_train shape: {Y_test.shape}')

X_train shape: (60000, 784)
X_test shape: (10000, 784)
Y_train shape: (60000,)
Y_train shape: (10000,)


## Activation Function 
- sigmoid & relu : hidden layer activation function 
- softmax : output layer activation function 

In [4]:
class sigmoid:
    # sigmoid 함수를 작성하세요 
    def forward(x):
        return  1 / (1+np.exp(-x))
    
    # sigmoid 함수의 미분을 작성하세요
    def backward(x):
        return x * (1 - x)

In [5]:
class relu:
    # relu 함수를 작성하세요
    def forward(x):
        return np.maximum(0, x) 
    
    # relu 함수의 미분을 작성하세요
    def backward(x):
        return np.where(x > 0, 1, 0)

In [6]:
class softmax:
    def forward(z):
        y = []
        for zi in z:
            c = np.max(zi)
            exp_zi = np.exp(zi-c)
            sum_exp_zi = np.sum(exp_zi)
            yi = exp_zi / sum_exp_zi
            y.append(yi)

        return np.array(y)
    
    def backward(p, y) :
        dp = p.copy()
        for dpi, yi in zip(dp, y):
            for k in range(dp.shape[1]):
                if k == yi :
                    dpi[k] -= 1
        return dp

## Loss Function

In [7]:
def cross_entropy(p, y):
    loss = []
    for pi, yi in zip(p, y):
        for k in range(p.shape[1]):
            if k == yi:
                loss.append((-1) * (np.log(pi[k] + 1e-8))) 
    return loss

## Layer

In [8]:
class Layer:
    def __init__(self, input_size, output_size, std=1e-4) :
        self.input_size = input_size
        self.output_size = output_size
        self.bias = np.random.randn(output_size)
        self.weight = np.random.randn(input_size, output_size)*std

## Neural Network
- 각 메소드와 변수들의 역할을 주석으로 달아주세요! 

In [9]:
class CustomNet:
    # CustomNet을 선언할 때 생성되는 값들입니다.
    def __init__(self, lr=0.0001, epoch=500, batch_size=200):
        self.lr = lr
         # 데이터셋에 대하여 모델이 학습을 반복할 횟수
        self.epoch = epoch  
         # 모델이 학습하고 가중치를 업데이트하는 데이터 사이즈 
        self.batch_size = batch_size
        
        self.loss_function = cross_entropy 
        # Layer를 담고있는 배열
        self.layers = []
        # 활성화 함수를 담고있는 배열
        self.activations = [softmax]
        # 레이어의 노드의 수에 대한 정보
        self.nodes = []
    
    # Layer를 추가할 때 호출합니다
    def addLayer(self, Layer): 
        self.layers.append(Layer) 
        if not self.nodes: 
            self.nodes.append(np.zeros(Layer.input_size))
        self.nodes.append(np.zeros(Layer.output_size)) # output 노드들의 수를 추가해가며 Layer의 노드 수를 담는다
        
    # Activation Function을 추가할 때 호출합니다
    def addActivation(self, Activation):
        tmp = self.activations.pop()  # 가장 뒷 단에 있는 활성 함수를 추출한다(이 경우 Softmax)
        self.activations.append(Activation) # 추가한 활성함수를 추가한다
        self.activations.append(tmp) 
        
    # 순전파 함수
    def _forward(self, X):
        self.nodes[0] = X.copy() # 입력받은 X로 첫번째 노드를 초기화한다
        output = X.copy() 
        for i in range(len(self.layers)): 
            Layer = self.layers[i]  # 반복에 사용할 개별 레이어 지정
            Activation = self.activations[i]  # 레이어별 활성화 함수 지정
            output = np.dot(self.nodes[i], Layer.weight) # 노드의 값과 가중치를 행렬곱해서 결괏값 산출
            output = output+ Layer.bias 
            output = Activation.forward(output) 
            self.nodes[i+1] = output  # 다음번 노드의 결괏값으로 초기화
        return output   
    
    # 역전파 함수
    def _backward(self, X, output, y) :
        for i in reversed(range(len(self.layers))): # layer의 길이의 반대순으로 실행
            a = self.nodes[i+1] 
            Layer = self.layers[i] 
            Activation = self.activations[i] 
            # 은닉층별로 error를 계산
            if i+1 == len(self.layers): 
                error = Activation.backward(output, y)
            else:
                error *= Activation.backward(a)
            Layer.weight -= np.dot(error.T, self.nodes[i]).T*self.lr/X.shape[0] # 에러와 값과 learingrate를 이용해 가중치 업데이트
            Layer.bias -= error.sum(axis=0)*self.lr/X.shape[0]
            error = np.dot(error, Layer.weight.T) 
            
    # Accrucy를 반환합니다
    def _accuracy(self, output, y):
        pre_p = np.argmax(output, axis=1)
        return np.sum(pre_p==y)/y.shape[0] 
    
    # 데이터셋에 모델을 fit할때 호출합니다
    def fit(self, X, y, val_X, val_y):
        history = {'val_acc': [],'val_loss': []}
        N = X.shape[0]
        for i in range(self.epoch):  # epoch 수 만큼 실행
            for j in range(N//self.batch_size):  # mini batch 크기만큼 진행
                batch_mask = np.random.choice(N, self.batch_size)
                X_batch = X[batch_mask] 
                y_batch = y[batch_mask] 
                output = self._forward(X_batch) 
                self._backward(X_batch, output, y_batch)
            
            #accuracy와 loss를 기록해둡시다
            output = self._forward(val_X) 
            history["val_acc"].append(self._accuracy(output, val_y)) 
            history["val_loss"].append(sum(self.loss_function(output, val_y))) 
            
            #중간중간 기록을 찍어볼 때 사용. 적절히 조절해 쓰세요
            if i % 10 == 0:
                print(i, "test accuracy :", history["val_acc"][-1])
                print(i, "test loss     :", history["val_loss"][-1])
        return history

## Customizing
- Network parameter, Layer architecture, Activation function .. 등등 다양한 하이퍼파라미터를 커스터마이징하여 높은 성능에 도달해 봅시다! 

In [10]:
# 하이퍼파라미터를 적절히 조절해 뉴럴넷을 선언하세요
nn = CustomNet(lr=0.005, epoch=200, batch_size=400)

# 원하는 만큼 층과 활성화 함수를 쌓아 주세요. 기본적으로 2Layer를 예시로 적어드립니다
nn.addLayer(Layer(784,100))
nn.addActivation(sigmoid)
nn.addLayer(Layer(100,10))

In [11]:
# 선언한 뉴럴넷의 구조입니다
for layer in nn.layers:
    print(layer.weight.shape, layer.bias.shape)

(784, 100) (100,)
(100, 10) (10,)


In [12]:
history = nn.fit(X_train, Y_train, X_test, Y_test)

0 test accuracy : 0.098
0 test loss     : 23136.86248492996
10 test accuracy : 0.114
10 test loss     : 22738.175735771565
20 test accuracy : 0.1311
20 test loss     : 22329.32209892325
30 test accuracy : 0.2325
30 test loss     : 21526.29653169128
40 test accuracy : 0.2662
40 test loss     : 20276.778378053743
50 test accuracy : 0.3464
50 test loss     : 18998.02543826738
60 test accuracy : 0.4258
60 test loss     : 17551.90923923011
70 test accuracy : 0.4976
70 test loss     : 15523.772205645011
80 test accuracy : 0.5434
80 test loss     : 13708.892892747896
90 test accuracy : 0.597
90 test loss     : 12302.380887691623
100 test accuracy : 0.6635
100 test loss     : 10987.419705969709
110 test accuracy : 0.7129
110 test loss     : 9774.753678149082
120 test accuracy : 0.7595
120 test loss     : 8736.280070905617
130 test accuracy : 0.7872
130 test loss     : 7869.7936651533755
140 test accuracy : 0.8128
140 test loss     : 7145.418718571724
150 test accuracy : 0.8299
150 test loss   

## Accuracy, Loss Visualization
- 자유롭게 Accuracy나 Loss를 시각화하여 확인하고 결과를 확인해 보세요! 